In [ ]:
# default_exp core

# pandas temporal query language





In [0]:
#export

import re
import numpy as np
import pandas as pd
from functools import singledispatch

# General helper functions

## Info

In [0]:
#export
class Info():
  """
  A class to store information about the data and results from analysis
  """
  def __init__(self):
      self.evaluated = {}

## memory

In [0]:
#export
def memory(info, func, expr):
  """
  checks if the function has been called with the same argument previously and
  if so, returns the same results instead of running the function again

  args:
    -
  """
  rows=None
  if info:
    if func in info.evaluated:
      if expr in info.evaluated[func]:
        rows = info.evaluated[func][expr]
    else:
      info.evaluated[func] = {}
  else:
    info = Info()
    info.evaluated[func] = {}
  return info, rows

## listify

In [0]:
#export
def listify(string_or_list):
    """
    return a list if the input is a string, if not: returns the input as it was

    Args:
        string_or_list (str or any):

    Returns:
        A list if the input is a string, if not: returns the input as it was

    Note:
        - allows user to use a string as an argument instead of single lists
        - cols='icd10' is allowed instead of cols=['icd10']
        - cols='icd10' is transformed to cols=['icd10'] by this function

    """
    if isinstance(string_or_list, str):
        string_or_list = [string_or_list]
    return string_or_list

## unique

In [0]:
#export
# A function to identify all unique values in one or more columns
# with one or multiple codes in each cell

def unique(df, cols=None, sep=None, all_str=True):
  """
  Lists unique values from one or more columns

  sep (str): separator if cells have multiple values
  all_str (bool): converts all values to strings

  unique(df=df, cols='inpatient', sep=',')
  """
  # if no column(s) are specified, find unique values in whole dataframe
  if cols==None:
    cols=list(df.columns)
  cols = listify(cols)

  # multiple values with separator in cells
  if sep:
    all_unique=set()

    for col in cols:
      new_unique = set(df[col].str.cat(sep=',').split(','))
      all_unique.update(new_unique)
  # single valued cells
  else:
    all_unique = pd.unique(df[cols].values.ravel('K'))

  # if need to make sure all elements are strings without surrounding spaces
  if all_str:
    all_unique=[str(value).strip() for value in all_unique]

  return all_unique

In [0]:
#unique(df=df, cols='codes', sep=',')

## del dot and zero

In [0]:
#export
def del_dot(code):
  if isinstance(code, str):
    return code.replace('.','')
  else:
    codes = [c.replace('.','') for c in code]
  return codes

def del_zero(code, left=True, right=False):
  if isinstance(codes, str):
    codes=[code]
  if left:
    codes = [c.lstrip('0') for c in code]
  if right:
    codes = [c.rstrip('0') for c in code]
  if isinstance(code, str):
    codes=codes[0]
  return codes

# Notations

## expand hyphen

In [0]:
#export
# function to expand a string like 'K51.2-K53.8' to a list of codes

# Need regex to extract the number component of the input string

# The singledispach decorator enables us to have the same name, but use
# different functions depending on the datatype of the first argument.
#
# In our case we want one function to deal with a single string input, and
# another to handle a list of strings. It could all be handled in a single
# function using nested if, but singledispatch makes it less messy and more fun!


# Here is the main function, it is just the name and an error message if the
# argument does not fit any of the inputs that wil be allowed

@singledispatch
def expand_hyphen(expr):
  """
  Expands codes expression(s) that have hyphens to list of all codes

  Args:
      code (str or list of str): String or list of strings to be expanded

  Returns:
      List of strings

  Examples:
      expand_hyphen('C00*-C26*')
      expand_hyphen('b01.1*-b09.9*')
      expand_hyphen('n02.2-n02.7')
      expand_hyphen('c00*-c260')
      expand_hyphen('b01-b09')
      expand_hyphen('b001.1*-b009.9*')
      expand_hyphen(['b001.1*-b009.9*', 'c11-c15'])
  Note:
      Unequal number of decimals in start and end code is problematic.
      Example: C26.0-C27.11 will not work since the meaning is not obvious:
      Is the step size 0.01? In which case C27.1 will not be included, while
      C27.10 will be (and traing zeros can be important in codes)
  """
  raise ValueError('The argument must be a string or a list')

# register the function to be used if the input is a string
@expand_hyphen.register(str)
def _(expr):
    # return immediately if nothing to expand
    if '-' not in expr:
      return [expr]

    lower, upper = expr.split('-')

    lower=lower.strip()

    # identify the numeric component of the code
    lower_str = re.search("\d*\.\d+|\d+", lower).group()
    upper_str = re.search("\d*\.\d+|\d+", upper).group()
    # note: what about european decimal notation?
    # also note: what if multiple groups K50.1J8.4-etc


    lower_num = int(lower_str.replace('.',''))
    upper_num = int(upper_str.replace('.','')) +1

    if upper_num<lower_num:
      raise ValueError('The start code cannot have a higher number than the end code')

    # remember length in case of leading zeros
    length = len(lower_str)

    nums = range(lower_num, upper_num)

    # must use integers in a loop, not floats
    # which also means that we must multiply and divide to get decimal back
    # and take care of leading and trailing zeros that may disappear
    if '.' in lower_str:
      lower_decimals = len(lower_str.split('.')[1])
      upper_decimals = len(upper_str.split('.')[1])
      if lower_decimals==upper_decimals:
        multiplier = 10**lower_decimals
        codes = [lower.replace(lower_str, format(num /multiplier, f'.{lower_decimals}f').zfill(length)) for num in nums]
      # special case: allow k1.1-k1.123, but not k.1-k2.123 the last is ambigious: should it list k2.0 only 2.00?
      elif (lower_decimals<upper_decimals) & (upper_str.split('.')[0]==lower_str.split('.')[0]):
        from_decimal = int(lower_str.split('.')[1])
        to_decimal = int(upper_str.split('.')[1]) +1
        nums = range(from_decimal, to_decimal)
        decimal_str = '.'+lower.split('.')[1]
        codes = [lower.replace(decimal_str, '.'+str(num)) for num in nums]
      else:
        raise ValueError('The start code and the end code do not have the same number of decimals')
    else:
        codes = [lower.replace(lower_str, str(num).zfill(length)) for num in nums]
    return codes


# register the function to be used if if the input is a list of strings
@expand_hyphen.register(list)
def _(expr):
  extended = []
  for word in expr:
    extended.extend(expand_hyphen(word))
  return extended

## expand star

In [0]:
#export
# A function to expand a string with star notation (K50*)
# to list of all codes starting with K50

@singledispatch
def expand_star(code, all_codes=None):
  """
  Expand expressions with star notation to a list of all values with the specified pattern

  Args:
    expr (str or list): Expression (or list of expressions) to be expanded
    all_codes (list) : A list of all codes

  Examples:
    expand_star('K50*', all_codes=icd9)
    expand_star('K*5', all_codes=icd9)
    expand_star('*5', all_codes=icd9)

  """
  raise ValueError('The argument must be a string or a list')

@expand_star.register(str)
def _(code, all_codes=None):
  # return immediately if there is nothing to expand
  if '*' not in code:
    return [code]

  start_str, end_str = code.split('*')

  if start_str and end_str:
    codes = {code for code in all_codes if (code.startswith(start_str) & code.endswith(end_str))}

  if start_str:
    codes = {code for code in all_codes if code.startswith(start_str)}

  if end_str:
    codes = {code for code in all_codes if code.endswith(end_str)}

  return sorted(list(codes))

@expand_star.register(list)
def _(code, all_codes=None):

  expanded=[]
  for star_code in code:
    new_codes = expand_star(star_code, all_codes=all_codes)
    expanded.extend(new_codes)

  # uniqify in case some overlap
  expanded = list(set(expanded))

  return sorted(expanded)

## expand colon

In [0]:
#export
# function to get all codes in a list between the specified start and end code
# Example: Get all codes between K40:L52

@singledispatch
def expand_colon(code, all_codes=None):
  raise ValueError('The argument must be a string or a list')

@expand_colon.register(str)
def _(code, all_codes=None):
  """
  Expand expressions with colon notation to a list of complete code names
  code (str or list): Expression (or list of expressions) to be expanded
  all_codes (list or array) : The list to slice from

  Examples
    K50:K52
    K50.5:K52.19
    A3.0:A9.3

  Note: This is different from hyphen and star notation because it can handle
  different code lengths and different number of decimals

  """
  if ':' not in code:
    return [code]

  startstr, endstr = code.split(':')

  # remove spaces
  startstr = startstr.strip()
  endstr =endstr.strip()

  # find start and end position
  startpos = all_codes.index(startstr)
  endpos = all_codes.index(endstr) + 1

  # slice list
  expanded = all_codes[startpos:endpos+1]

  return expanded


@expand_colon.register(list)
def _(code, all_codes=None, regex=False):
  expanded=[]

  for cod in code:
    new_codes = expand_colon(cod, all_codes=all_codes)
    expanded.extend(new_codes)

  return expanded

## expand regex

In [0]:
#export
# Return all elements in a list that fits a regex pattern

@singledispatch
def expand_regex(code, all_codes):
  raise ValueError('The argument must be a string or a list of strings')

@expand_regex.register(str)
def _(code, all_codes=None):
  code_regex = re.compile(code)
  expanded = {code for code in all_codes if code_regex.match(code)}
  # uniqify
  expanded = list(set(expanded))
  return expanded

@expand_regex.register(list)
def _(code, all_codes):
  expanded=[]

  for cod in code:
    new_codes = expand_regex(cod, all_codes=all_codes)
    expanded.extend(new_codes)

  # uniqify in case some overlap
  expanded = sorted(list(set(expanded)))

  return expanded

## expand code

In [0]:
#export
@singledispatch
def expand_code(code, all_codes=None,
                hyphen=True, star=True, colon=True, regex=False,
                drop_dot=False, drop_leading_zero=False,
                sort_unique=True):
  raise ValueError('The argument must be a string or a list of strings')

@expand_code.register(str)
def _(code, all_codes=None,
      hyphen=True, star=True, colon=True, regex=False,
      drop_dot=False, drop_leading_zero=False,
      sort_unique=True):
  #validating input
  if (not regex) and (':' in code) and (('-' in code) or ('*' in code)):
    raise ValueError('Notation using colon must start from and end in specific codes, not codes using star or hyphen')

  if regex:
    codes = expand_regex(code, all_codes=all_codes)
    return codes

  if drop_dot:
    code = del_dot(code)

  codes=[code]

  if hyphen:
    codes=expand_hyphen(code)
  if star:
    codes=expand_star(codes, all_codes=all_codes)
  if colon:
    codes=expand_colon(codes, all_codes=all_codes)

  if sort_unique:
    codes = sorted(list(set(codes)))

  return codes

@expand_code.register(list)
def _(code, all_codes=None, hyphen=True, star=True, colon=True, regex=False,
      drop_dot=False, drop_leading_zero=False,
      sort_unique=True):

  expanded=[]

  for cod in code:
    new_codes = expand_code(cod, all_codes=all_codes, hyphen=hyphen, star=star, colon=colon, regex=regex, drop_dot=drop_dot, drop_leading_zero=drop_leading_zero)
    expanded.extend(new_codes)

  # uniqify in case some overlap
  expanded = list(set(expanded))

  return sorted(expanded)

## expand columns

In [0]:
#export
def expand_columns(expr, all_columns=None, df=None, star=True,
                   hyphen=True, colon=True, regex=None, info=None):
    """
    Expand columns with special notation to their full column names

    """
    notations = '* - :'.split()
    # return immediately if not needed
    if not any(symbol in expr for symbol in notations):
      return [expr]

    # get a list of columns of it is only implicity defined by the df
    # warning: may depreciate this, require explicit all_columns
    if df & (not all_columns):
      all_columns=list(df.columns)

    if regex:
      cols = [col for col in all_columns if re.match(regex, expr)]
    else:
      if hyphen:
        cols = expand_hyphen(expr)
      if star:
        cols = expand_star(expr, all_codes=all_columns)
      if colon:
        cols = expand_colon(expr, all_codes=all_columns)

    return cols

# More helper functions


## get rows

In [0]:
#export
# mark rows that contain certain codes in one or more colums
def get_rows(df, codes, cols=None, sep=None, pid='pid', info=None, fix=True):
  """
  Make a boolean series that is true for all rows that contain the codes

  Args
    df (dataframe or series): The dataframe with codes
    codes (str, list, set, dict): codes to be counted
    cols (str or list): list of columns to search in
    sep (str): The symbol that seperates the codes if there are multiple codes in a cell
    pid (str): The name of the column with the personal identifier

  """

  # check if evaluated previously
  info, rows = memory(info=info, func = 'get_rows', expr=codes)
  if rows:
    return rows

  # check if codes and columns need to be expanded (needed if they use notation)
  if fix:
    # do this when if cols exist, but if it does not ...
    cols = expand_columns(expr=cols, all_columns=list(df.columns), info=info)
    all_codes = sorted(unique(df=df, cols=cols, sep=sep))
    codes = expand_code(codes, all_codes=all_codes)

  # codes and cols should be lists
  codes = listify(codes)
  cols = listify(cols)

  # approach depends on whether we have multi-value cells or not
  # if sep exist, then have multi-value cells
  if sep:
    # have multi-valued cells
    # note: this assumes the sep is a regex word delimiter
    codes = [rf'\b{code}\b' for code in codes]
    codes_regex = '|'.join(codes)

    # starting point: no codes have been found
    # needed since otherwise the function might return None if no codes exist
    rows = pd.Series(False*len(df),index=df.index)

   # loop over all columns and mark when a code exist
    for col in cols:
      rows=rows | df[col].str.contains(codes_regex, na=False)

  # if not multi valued cells
  else:
    mask = df[cols].isin(codes)
    rows = mask.any(axis=1)
  return rows

## make codes

In [0]:
#export
def make_codes(n, letters=26, numbers=100, seed=False):
  """
  Generate a dataframe with a column of random codes

  Args:
    letters (int): The number of different letters to use
    numbers (int): The number of different numbers to use

  Returns
    A dataframe with a column with one or more codes in the rows

  """
  # each code is assumed to consist of a letter and a number
  alphabet = list('abcdefghigjklmnopqrstuvwxyz')
  letters=alphabet[:letters+1]

  # make random numbers same if seed is specified
  if seed:
    np.random.seed(0)

  # determine the number of codes to be drawn for each event
  n_codes=np.random.negative_binomial(1, p=0.3, size=n)
  # avoid zero (all events have to have at least one code)
  n_codes=n_codes+1

  # for each event, randomly generate a the number of codes specified by n_codes
  codes=[]
  for i in n_codes:
      diag = [np.random.choice(letters).upper()+
              str(int(np.random.uniform(low=1, high=numbers)))
              for num in range(i)]

      code_string=','.join(diag)
      codes.append(code_string)

  # create a dataframe based on the list
  df=pd.DataFrame(codes)
  df.columns=['code']

  return df

In [0]:
make_codes(10)

## make data

In [0]:
#export
def make_data(n, letters=26, numbers=100, seed=False):
  """
  Generate a dataframe with a column of random codes

  Args:
    letters (int): The number of different letters to use
    numbers (int): The number of different numbers to use

  Returns
    A dataframe with a column with one or more codes in the rows

  """
  pid = range(n)
  df_person=pd.DataFrame(index = pid)

  #female = np.random.binomial(1, 0.5, size =n)
  gender = np.random.choice(['male', 'female'], size=n)
  region = np.random.choice(['north', 'south', 'east', 'west'], size=n)
  birth_year = np.random.randint(1920, 1980, size=n)
  birth_month = np.random.randint(1,12, size=n)
  birth_day = np.random.randint(1,28, size=n) # ok, I know!
  events_per_year = np.random.poisson(1, size=n)
  years = 2020 - birth_year
  events = years * events_per_year
  events = np.where(events==0,1,events)
  events = events.astype(int)
  all_codes=[]
  codes = [all_codes.extend(make_codes(n=n, letters=letters,
                                       numbers=numbers,
                                       seed=seed)['code'].tolist())
          for n in events]

  days_alive = (2020 - birth_year) *365

  days_and_events = zip(days_alive.tolist(), events.tolist())
  all_days=[]
  days_after_birth = [all_days.extend(np.random.randint(0, max_day, size=n)) for max_day, n in days_and_events]
  pid_and_events = zip(list(pid), events.tolist())
  all_pids=[]
  pids = [all_pids.extend([p+1]*e) for p, e in pid_and_events]

  df_events = pd.DataFrame(index=all_pids)
  df_events['codes'] = all_codes
  df_events['days_after'] = all_days

  #df_person['female'] = female
  df_person['gender'] = gender

  df_person['region'] = region
  df_person['year'] = birth_year
  df_person['month'] = birth_month
  df_person['day'] = birth_day
  df = df_events.merge(df_person, left_index=True, right_index=True)
  df['birth_date'] = pd.to_datetime(df[['year', 'month', 'day']])
  df['event_date'] = df['birth_date'] + pd.to_timedelta(df.days_after, unit='d')
  del df['month']
  del df['day']
  del df['days_after']
  df['pid'] = df.index
  df.index_name = 'pid_index'
  df=df[['pid', 'gender', 'birth_date', 'event_date', 'region', 'codes']]
  # include deaths too?
  return df

In [0]:
#df = make_data(n=1000)

In [0]:
#df

In [0]:
#count_person('max 2 L35')
#count_person('x before y')

In [ ]:
#export
def extract_codes(df, codes, cols=None, sep=None, new_sep=',', na_rep='',
                  prefix=None, merge=False, out='bool', _fix=True, series=True, group=False):
    """
    Produce one or more columns with only selected codes

    Args:
        df (dataframe): Dataframe with events

        codes (string, list or dict): The codes for the disease

        cols (string, list): Name of columns where codes are located

        sep (string, default: None): Separator between codes in same cell (if exist)
            (If None, the function will infer the separator)

        pid (str, default: 'pid'): Name of column with the personal identification number

        codebook (list): User specified list of all possible or allowed codes

        merge (bool): Content of all columns is merged to one series # only if out='text'?

        group (bool): Star an other notation remain a single group, not split into individual codes

        out (string, ['text', 'category', 'bool' or 'int']): Datatype of output column(s)

    Notes:
        Can produce a set of dummy columns for codes and code groups.
        Can also produce a merged column with only extracted codes.
        Accept star notation.
        Also accepts both single value columns and columns with compound codes and separators
        Repeat events in same rows are only extracted once


    Example:
    to create three dummy columns, based on codes in icdmain column:

    >>> extract_codes(df=df,
    >>>          codes={'fracture' : 'S72*', 'cd': 'K50*', 'uc': 'K51*'},
    >>>          cols=['icdmain', 'icdbi'],
    >>>          merge=False,
    >>>          out='text')

    nb: problem with extract rows if dataframe is empty (none of the requested codes)
    """
    if _fix:
        df, cols = _to_df(df=df, cols=cols)
        codes, cols, allcodes, sep = _fix_args(df=df, codes=codes, cols=cols, sep=sep, group=group, merge=merge)

    subset = pd.DataFrame(index=df.index)

    for k, v in codes.items():
        rows = get_rows(df=df, codes=v, cols=cols, sep=sep, _fix=False)
        if out == 'bool':
            subset[k] = rows
        elif out == 'int':
            subset[k] = rows.astype(int)
        elif out == 'category':
            subset.loc[rows, k] = k
            subset[k] = subset[k].astype('category')
        else:
            subset[k] = na_rep
            subset.loc[rows, k] = k

    if (merge) and (out == 'bool'):
        subset = subset.astype(int).astype(str)

    new_codes = list(subset.columns)

    if (merge) and (len(codes) > 1):
        headline = ', '.join(new_codes)
        merged = subset.iloc[:, 0].str.cat(subset.iloc[:, 1:].values, sep=new_sep,
                                           na_rep=na_rep)  # strange .T.values seemed to work previouslyi but it should not have
        merged = merged.str.strip(',')
        subset = merged
        subset.name = headline
        if out == 'category':
            subset = subset.astype('category')

    # return a series if only one code is asked for (and also if merged?)
    if series and (len(codes) == 1):
        subset = subset.squeeze()

    return subset


In [ ]:
#export
def sankey_format(df, labels=None, normalize=False, dropna=False, threshold=0.01):
    """
    Format the dataframe so it is easy fo create a holoviews sankey figure

    labels=dict(bio_codes.values())
    import holoviews as hv
    hv.Sankey(t1).options(label_position='left')
    hv.extension('bokeh')
    t4=t1.copy()

    """
    a = df
    a = a.apply(lambda row: ' '.join(row))
    a = a.str.split(expand=True)

    a = a.replace(labels)
    for col in a.columns:
        a[col] = a[col] + ' (' + str(col + 1) + ')'


    if not dropna:
        a = a.fillna(f'No new')

    all_counts = {}
    for col in range(len(a.columns))[1:]:
        counts = a.groupby(a[col - 1])[col].value_counts(normalize=normalize)
        if normalize:
            counts = counts.mul(100).astype(int).fillna(0)

        counts.name = 'value'
        # counts = counts.rename(index=labels).reset_index()
        counts = counts.reset_index()
        counts.columns = ['source', 'target', 'value']

        all_counts[col] = counts
    t1 = pd.concat(all_counts, ignore_index=True)

    #if normalize:
    #    t1['value'] = t1['value'] / t1['value'].sum()

    t1 = t1[t1.source != 'No new']

    # a.groupby(1)[2].value_counts()
    return t1

In [ ]:
#export
def _format_codes(codes, merge=True):
    """
    Makes sure that the codes has the desired format: a dict with strings as
    keys (name) and a list of codes as values)

    Background: For several functions the user is allower to use strings
    when there is only one element in the list, and a list when there is
    no code replacement or aggregations, or a dict. To avoid (even more) mess
    the input is standardised as soon as possible in a function.

    Examples:
            codes = '4AB02'
            codes='4AB*'
            codes = ['4AB02', '4AB04', '4AC*']
            codes = ['4AB02', '4AB04']
            codes = {'tumor' : 'a4*', 'diabetes': ['d3*', 'd5-d9']}
            codes = 'S72*'
            codes = ['K50*', 'K51*']

            _format_codes(codes, merge=False)

    TODO: test for correctness of input, not just reformat (is the key a str?)
    """
    codes = _listify(codes)

    # treeatment of pure lists depends on whether special classes should be treated as one merged group or separate codes
    # exmple xounting of Z51* could mean count the total number of codes with Z51 OR a shorthand for saying "count all codes starting with Z51 separately
    # The option "merged, enables the user to switch between these two interpretations

    if isinstance(codes, list):
        if merge:
            codes = {'_'.join(codes): codes}
        else:
            codes = {code: [code] for code in codes}

    elif isinstance(codes, dict):
        new_codes = {}
        for name, codelist in codes.items():
            if isinstance(codelist, str):
                codelist = [codelist]
            new_codes[name] = codelist
        codes = new_codes

    return codes

In [ ]:
#export
def _expand_regex(expr, full_list):
    exprs = _listify(expr)

    expanded = []

    if isinstance(full_list, pd.Series):
        pass
    elif isinstance(full_list, list):
        unique_series = pd.Series(full_list)
    elif isinstance(full_list, set):
        unique_series = pd.Series(list(full_list))

    for expr in exprs:
        match = unique_series.str.contains(expr)
        expanded.extend(unique_series[match])
    return expanded

In [ ]:
#export
def _reverse_dict(dikt):
    new_dict = {}
    for name, codelist in dikt.items():
        codelist = _listify(codelist)
        new_dict.update({code: name for code in codelist})
    return new_dict


def persons_with(df,
                 codes,
                 cols,
                 pid='pid',
                 sep=None,
                 merge=True,
                 first_date=None,
                 last_date=None,
                 group=False,
                 _fix=True):
    """
    Determine whether people have received a code

    Args:
        codes (list or dict): codes to mark for
            codes to search for
                - if list: each code will represent a column
                - if dict: the codes in each item will be aggregated to one indicator
            cols (str or list of str): Column(s) with the codes
            pid (str): colum with the person identifier
            first_date (str): use only codes after a given date
                the string either represents a date (same for all individuals)
                or the name of a column with dates (may be different for different individuals)
            last_date (str): only use codes after a given date
                the string either represents a date (same for all individuals)
                or the name of a column with dates (may be different for different individuals)

    Returns:
        Series or Dataframe


    Examples:
        fracture = persons_with(df=df, codes='S72*', cols='icdmain')
        fracture = persons_with(df=df, codes={'frac':'S72*'}, cols='icdmain')

    Todo:
        - function may check if pid_index is unique, in which it does not have to aggregate
        - this may apply in general? functions that work on event data may then also work on person level data
        - allow user to input person level dataframe source?
    """
    sub = df

    if _fix:
        df, cols = _to_df(df=df, cols=cols)
        codes, cols, allcodes, sep = _fix_args(df=df, codes=codes, cols=cols, sep=sep, merge=merge, group=group)
        rows = get_rows(df=df, codes=allcodes, cols=cols, sep=sep, _fix=False)
        sub = df[rows]

    df_persons = sub.groupby(pid)[cols].apply(lambda s: pd.unique(s.values.ravel()).tolist()).astype(str)

    # alternative approach, also good, and avoids creaintg personal dataframe
    # but ... regeis is fast since it stopw when it finds one true code!
    #    c=df.icdbi.str.split(', ', expand=True).to_sparse()
    #    c.isin(['S720', 'I10']).any(axis=1).any(level=0)

    persondf = pd.DataFrame(index=df[pid].unique().tolist())
    for name, codes in codes.items():
        codes_regex = '|'.join(codes)
        persondf[name] = df_persons.str.contains(codes_regex, na=False)

    return persondf

In [ ]:
#export
def stringify_durations(df,
                        codes=None,
                        cols=None,
                        pid='pid',
                        step=120,
                        sep=None,

                        event_start='in_date',
                        event_end=None,
                        event_duration='ddd',

                        first_date=None,
                        last_date=None,
                        censored_date=None,

                        ncodes=None,

                        no_event='-',
                        time_sep='|',

                        merge=True,
                        info=None,
                        report=False):
    """
    Creates a string for each individual describing the time duration selected code events (example: a-, ad, --, a)

    Args:
        df: dataframe
        codes: codes to be used to mark an event
        cols: columns with the event codes
        pid: column with the personal identification number
        event_start: column containing the date for the event
        sep: the separator used between events if a column has multiple events in a cell
        keep_repeats: identical events after each other are reduced to one (if true)
        only_unique: deletes all events that have occurred previously for the individual (if true)

    Returns:
        series with a string that describes the events for each individual
    Example:

    >>> codes={'i' : ['4AB02', 'L04AB02'], 'a': ['4AB04','L04AB04']}
    >>> events=sa.stringify_durations(df=mdf, codes=codes, cols='codes',
    event_start='date', first_date=None, sep=',', merge=True)

    >>> codes={'i' : ['4AB02', 'L04AB02'], 'a': ['4AB04','L04AB04']}
    >>> codes={'i' : ['L04*'], 'b': ['4AB04','L04AB04']}


    >>> codes = {'i':'L01BB02 L04AX03 L01BA01 L04AD01 L04AD02 L04AA06'.split(),
                 'b':'L04AB02 L04AB04 L04AB06 L04AA33 L04AC05 L04AA23'.split()}


    >>> events=sa.stringify_durations(df=mdf, codes=codes, cols='codes',
    event_start='date', first_date=None, sep=',', merge=False, step=100)

    >>> codes={'L04A*' : 'i', 'L04AB*' : 'a', 'H02*' : 'c'}
    >>> pr=pr.set_index('pid_index')
    >>> pr['first_date'] = pr.groupby('pid')['date'].min()
    >>> events=stringify_durations(df=df, codes=codes, col='ncmpalt', start='start_date', first_date='first', dataset_end_date="01-01-2018")


    background
        to identify treatment patters, first stringify each treatment,
        then aggregate the different treatments to one string
        each "cell" in the string (separated by sep) represent one time unit
        the time unit can be further aggregated to reduce the level of detail

    example output (one such row for each person)
        a---s, a---, ai-s, a---, ----

        Interpretation: A person with event a and s in first time perod, then a only in second,
        the a, i and s in the third, a only in fourth and no events in the last

    purpose
        examine typical treatment patterns and correlations
        use regex or other string operations on this to get statistcs
        (time on first line of treatment, number of switches, stops)

    """
    # drop rows with missing observations in required variables

    if report:
        obs = len(df)
        npid = df[pid].nunique()
        if isinstance(codes, dict):
            allcodes = _get_allcodes(codes)
        elif isinstance(codes, str):
            allcodes = _listify(codes)
        # todo: also possible notational codes! better eliminate this?
        rows = get_rows(df=df, codes=allcodes, cols=cols, sep=sep)
        code_obs = len(df[rows])
        code_npid = df[rows][pid].nunique()

    df = df.dropna(subset=[pid, event_start])  # also drop if codes is missing

    if event_end:
        df = df.dropna(subset=[event_end])
    elif event_duration:
        df = df.dropna(subset=[event_duration])
        if df[event_duration].min() < 0:
            print('Error: The specified duration column contains negative values. They are dropped')
            df = df[df[event_duration] >= 0]
    else:
        print('Error: Either event_end or event_duration has to be specified.')

    # find default min and max dates
    # will be used as starting points for the string
    # if first_date and last_date are not specified
    min_date = df[event_start].min()
    max_date = df[event_start].max()

    # drop rows outside specified time period of interest
    if first_date:
        if first_date in df.columns:
            df = df[df[event_start] >= df[first_date]]
        elif isinstance(first_date, dict):
            pass
        else:
            # if first_date is not a column name, it is assumed to be a date
            try:
                min_date = pd.to_datetime(first_date)
                df = df[df[event_start] >= min_date]
            except:
                print(
                    'Error: The first_date argument has to be on of: None, a dict, a column name or a string that represents a date')

    if last_date:
        if last_date in df.columns:
            df = df[df[event_start] >= df[last_date]]
        elif isinstance(last_date, dict):
            pass
        else:
            try:
                max_date = pd.to_datetime(last_date)
                df = df[df[event_start] <= max_date]
            except:
                print(
                    'Error: The last_date argument has to be on of: None, a dict, a column name or a string the represents a date')

    # note an individual min date cannot be before overall specified min date
    # should raise error if user tries this
    # same with max: individual cannot be larger than overall

    max_length_days = (max_date - min_date).days
    max_length_steps = int(max_length_days / step)

    # if codes are not specified, use the five most common codes
    if not codes:
        cols = _expand_cols(_listify(cols))
        if not ncodes: ncodes = 4
        codes = count_codes(df=df, cols=cols, sep=sep).sort_values(ascending=False)[:ncodes]

    # fix formatting of input (make list out of a string input and so on)
    codes, cols, allcodes, sep = _fix_args(df=df, codes=codes, cols=cols, sep=sep)

    # get the rows that contain the relevant codes
    rows = get_rows(df=df, codes=allcodes, cols=cols, sep=sep, _fix=False)
    subset = df[rows].copy()  # maybe use .copy to avoid warnings? but takes time and memory
    subset = subset.set_index(pid, drop=False)
    subset.index.name = 'pid_index'
    subset = subset.sort_values([pid, event_start])

    if report:
        sub_obs = len(subset)
        sub_npid = subset[pid].nunique()

    # find start and end position of each event (number of steps from overall min_date)
    # to do: do not use those column names (may overwrite original names), use uuid names?
    subset['start_position'] = (subset[event_start] - min_date).dt.days.div(step).astype(int)

    if event_end:
        subset['end_position'] = (subset[event_end] - min_date).dt.days.div(step).astype(int)
    elif event_duration:
        subset['end_date'] = subset[event_start] + pd.to_timedelta(subset[event_duration].astype(int), unit='D')
        subset['end_position'] = (subset['end_date'] - min_date).dt.days.div(step).astype(int)

    # to do: may allow duration dict?
    # for instance: some drugs last 15 days, some drugs last 25 days . all specified in a dict

    # create series with only the relevant codes for each person and position
    code_series = extract_codes(df=subset.set_index([pid, 'start_position', 'end_position']),
                                codes=codes,
                                cols=cols,
                                sep=sep,
                                new_sep=',',
                                merge=False,
                                out='text',
                                _fix=False)

    unique_codes = list(code_series.columns)

    code_series = pd.melt(code_series.reset_index(),
                          id_vars=['pid', 'start_position', 'end_position'],
                          value_vars=unique_codes)

    # drop duplicates (same type of even in same period for same individual)
    code_series = code_series.drop_duplicates().set_index(pid, drop=False)
    code_series.index.name = 'pid_index'
    ## make dict with string start and end positions for each individual
    # explanation:
    # the string is first made marking events in positions using calendar time
    # but often we want the end result to be strings that start at specified
    # individual dates, and not the same calendar date for all
    # for instance it is often useful to start the string at the date the
    # person receives a diagnosis
    # same with end of string: strings may end when a patient dies
    # user can specify start and end dates by pointing to columns with dates
    # or they may specify an overall start and end date
    # if individual dates are specified, the long string based on calendar
    # time is sliced to include only the relevant events

    if first_date:
        # if a column is specified
        if first_date in subset.columns:
            start_date = subset.groupby(pid)[first_date].first().dropna().to_dict()
        # do nothing if a dict mapping pids to last_dates is already specified
        elif isinstance(first_date, dict):
            pass
        # if a single overall date is specified
        else:
            date = pd.to_datetime(first_date)
            start_date = {pid: date for pid in subset[pid].unique()}
        # convert start date to start position in string
        string_start_position = {pid: int((date - min_date).days / step)
                                 for pid, date in start_date.items()}

    if last_date:
        if last_date in subset:
            end_date = subset.groupby(pid)[last_date].first().dropna().to_dict()
        # do nothing if a dict mapping pids to last_dates is already specified
        elif isinstance(last_date, dict):
            pass
        else:
            date = pd.to_datetime(last_date)
            end_date = {pid: date for pid in subset[pid].unique()}
        # convert date to position in string
        string_end_position = {pid: (date - min_date).dt.days.div(step).astype(int)
                               for pid, date in end_date.items()}

        # takes dataframe for an individual and makes a string with the events

    def make_string(events):
        # get pid of individual (required to find correct start and end point)
        person = events.index[0]

        # make a list of maximal length with no events
        event_list = [no_event] * (max_length_steps + 1)

        from_to_positions = tuple(zip(events['start_position'].tolist(), events['end_position'].tolist()))

        # loop over all events the individual has and put code in correct pos.
        for pos in from_to_positions:
            event_list[pos[0]:pos[1]] = code
        event_string = "".join(event_list)

        # slice to correct start and end of string (if specified)
        if first_date:
            event_string = event_string[string_start_position[person]:]
        if last_date:
            max_position = int((max_date - min_date).days / step)
            event_string = event_string[:-(max_position - string_end_position[person])]
        return event_string

    # new dataframe to store each string for each individual for each code
    string_df = pd.DataFrame(index=code_series[pid].unique())
    string_df.index.name = 'pid_index'

    # loop over each code, aggregate strong for each individual, store in df
    for code in unique_codes:
        code_df = code_series[code_series['value'].isin([code])] # maybe == is better (safer bco compounds + faster?)
        stringified = code_df.groupby(pid, sort=False).apply(make_string)
        string_df[code] = stringified

    if merge:
        string_df = interleave_strings(string_df, no_event=no_event, time_sep=time_sep)

    if report:
        final_obs = len(subset)
        final_npid = len(string_df)
        print(f"""
                                     events,  unique ids
              Original dataframe     {obs}, {npid} 
              Filter codes           {code_obs}, {code_npid}
              Filter missing         {sub_obs}, {sub_npid}
              Final result:          {final_obs}, {final_npid}""")
    return string_df

In [ ]:
#export
def _make_binary(df, cols=None, no_event=' ', time_sep='|', pad=False):
    if isinstance(df, pd.Series):
        name = df[col].name
        df=df.str.replace(no_event, '0')
        df=df.str.replace(name, '1')
    else:
        # if no cols are selected, use all cols
        if not cols:
            cols = list(df.columns)
        # replace event chars with 1 and no events with 0
        for col in cols:
            name = df[col].name
            df[col]=df[col].str.replace(no_event, '0')
            df[col]=df[col].str.replace(name, '1')
    return df

In [ ]:
#export
def interleave_strings(df, cols=None, time_sep="|", no_event=' ', agg=False):
    """
    Interleaves strings in two or more columns

    parameters
        cols : list of columns with strings to be interleaved
        nan : value to be used in place of missing values
        sep : seperator to be used between time periods
        agg : numeric, used to indicate aggregation of time scale
                default is 1

    background
        to identify treatment patters, first stringify each treatment,
        then aggregate the different treatments to one string
        each "cell" in the string (separated by sep) represent one time unit
        the time unit can be further aggregated to reduce the level of detail

    example output (one such row for each person)
        a---s, a---, ai-s, a---, ----

        Interpretation: A person with event a and s in first time perod, then a only in second,
        the a, i and s in the third, a only in fourth and no events in the last

    purpose
        examine typical treatment patterns and correlations
        use regex or other string operations on this to get statistcs
        (time on first line of treatment, number of switches, stops)

    """
    # if cols is not specified, use all columns in dataframe
    if not cols:
        cols = list(df.columns)

    if agg:
        for col in cols:
            df[col] = df[col].fillna(no_event)
            # find event symbol, imply check if all are missing, no events
            try:
                char = df[col].str.cat().strip().str.strip('-')[0]  # improvable?
            except:
                df[col] = (col.str.len() / agg) * no_event

            def aggregator(text, agg):
                missing = no_event * agg
                units = (text[i:i + agg] for i in range(0, len(text), agg))
                new_aggregated = (no_event if unit == missing else char for unit in units)
                new_str = "".join(new_aggregated)
                return new_str
        df[col] = df[col].apply(aggregator, agg=agg)

    if time_sep:
        interleaved = df[cols].fillna(no_event).apply(
            (lambda x: time_sep.join(
                "".join(i)
                for i in zip_longest(*x, fillvalue=no_event))),
            axis=1)
    else:
        interleaved = df[cols].fillna('-').apply(
            (lambda x: "".join(chain(*zip_longest(*x, fillvalue=no_event)))),
            axis=1)

    return interleaved

In [ ]:
#export
def left_justify(s, fill=' '):
    """
    after stringify, to make events at same time be in same position
    and no, not as crucial as left-pad!
    """
    nmax = s.apply(len).max()
    s = s.str.pad(width=nmax, side='right', fillchar=fill)
    return s

In [ ]:
#export
def overlay_strings(df, cols=None, sep=",", nan='-', collisions='x', interleaved=False):
    """
    overlays strings from two or more columns

    note
        most useful when aggregating a string for events that usually do not happen in the same time frame

    parameters
        cols : list of columns with strings to be interleaved
        nan : value to be used in place of missing values
        collisions: value to be used if there is a collision between events in a position


    background
        to identify treatment patters, first stringify each treatment,
        then aggregate the different treatments to one string
        each "cell" in the string (separated by sep) represent one time unit
        the time unit can be further aggregated to reduce the level of detail

    example output (one such row for each person)
        asaaa--s--aa-s-a

        Interpretation: A person with event a and s in first time perod, then a only in second,
        the a, i and s in the third, a only in fourth and no events in the last

    purpose
        examine typical treatment patterns and correlations
        use regex or other string operations on this to get statistcs
        (time on first line of treatment, number of switches, stops)

    todo
        more advanced handling of collisions
            - special symbols for different types of collisions
            - warnings (and keep/give info on amount and type of collisions)

    """
    # if cols is not specified, use all columns in dataframe
    if not cols:
        cols = list(df.columns)

    interleaved = df[cols].fillna('-').apply(
        (lambda x: "".join(chain(*zip_longest(*x, fillvalue='-')))),
        axis=1)
    step_length = len(cols)

    def event_or_collision(events):
        try:
            char = events.strip('-')[0]
        except:
            char = '-'
        n = len(set(events).remove('-'))
        if n > 1:
            char = 'x'
        return char

    def overlay_individuals(events):

        units = (events[i:i + step_length] for i in range(0, len(events), step_length))

        new_aggregated = (event_or_collision(unit) for unit in units)
        new_str = "".join(new_aggregated)
        return new_str

    interleaved.apply(overlay_individuals)

    return interleaved

In [ ]:
#export
def shorten(events, agg=3, no_event=' '):
    """
    create a new and shorter string with a longer time step

    parameters
        events: (str) string of events that will be aggregated
        agg: (int) the level of aggregation (2=double the step_length, 3=triple)
    """
    try:
        char = events.strip(no_event)[0]
    except:
        char = no_event
    units = (events[i:i + agg] for i in range(0, len(events), agg))
    new_aggregated = (no_event if unit == no_event else char for unit in units)
    new_str = "".join(new_aggregated)
    return new_str

In [ ]:
#export
def shorten_interleaved(text, agg=3, time_sep=',', no_event=' '):
    """
    text="a-si,a--i,a-s-,--si,---i,--s-"

    shorten_interleaved(c, agg=2)

    the original string must have a distinction between time_sep and no_event_sep
    (if not, could try to infer)
    """
    units = text.split(time_sep)
    ncodes = len(units[0])
    nunits = len(units)

    unitlist = [units[i:i + agg] for i in range(0, nunits, agg)]
    charlist = ["".join(aggunit) for aggunit in unitlist]
    unique_char = ["".join(set(chain(chars))) for chars in charlist]
    new_str = time_sep.join(unique_char)
    # ordered or sorted?
    # delete last if it is not full ie. not as many timee units in it as the others?
    # shortcut for all
    return new_str

In [ ]:
#export
def stringify_order(df, codes=None, cols=None, pid='pid', event_start='date',
                    sep=None, time_sep='', first_date=None, last_date=None, period=None, keep_repeats=True,
                    only_unique=False, _fix=True):
    """
    Creates a string for each individual describing selected code events in the order they occurred

    Args:
        df: dataframe
        codes: codes to be used to mark an event
        cols: columns with the event codes
        pid: column with the personal identification number
        event_start: column containing the date for the event
        sep: the separator used between events if a column has multiple events in a cell
        keep_repeats: identical events after each other are reduced to one (if true)
        only_unique: deletes all events that have occurred previously for the individual (if true)

    Returns:
        series with a string that describes the events for each individual



    Examples:

    >>> bio_codes= {'L04AA23': 'n', 'L04AA33': 'v', 'L04AB02': 'i', 'L04AB04': 'a','L04AB06': 'g', 'L04AC05': 'u'}

    >>> bio_codes={'e' : '4AB01', 'i' : '4AB02', 'a' : '4AB04'}

    >>> bio_codes={'i' : '4AB02', 'a' : '4AB04'}

    >>> bio_codes= {'n': ['L04AA23', '4AA23'],
                    'v': ['L04AA33', '4AA33'],
                    'i': ['L04AB02', '4AB02'],
                    'a': ['L04AB04', '4AB04'],
                    'g': ['L04AB06', '4AB06'],
                    'u': ['L04AC05', '4AC05']}


    >>> a=stringify_order(df=df, codes=bio_codes, cols='ncmpalt', pid='pid', event_start='start_date', sep=',', keep_repeats=True, only_unique=False)

    >>> a=sa.stringify_order(df=mdf, codes=bio_codes, cols='codes', pid='pid', first_date='first_ibd',
    event_start='date', sep=',', keep_repeats=False, only_unique=False, time_sep='', period=700)


    >>> bio_rows=get_rows(df=pr, codes=list(codes.keys()), cols='atc')
    >>> pr['first_bio']=pr[bio_rows].groupby('pid')['date'].min()

    >>> stringify_order(df=pr, codes=codes, cols='atc', pid='pid', event_date='date', sep=',')

    >>> stringify_order(df=pr, codes=bio_codes, cols='codes', pid='pid', event_date='date', sep=',')


    background
        to identify treatment patters, first stringify each treatment,
        then aggregate the different treatments to one string
        each "cell" in the string (separated by sep) represent one time unit
        the time unit can be further aggregated to reduce the level of detail

    example output (one such row for each person)
        a---s, a---, ai-s, a---, ----

        Interpretation: A person with event a and s in first time perod, then a only in second,
        the a, i and s in the third, a only in fourth and no events in the last

    purpose
        examine typical treatment patterns and correlations
        use regex or other string operations on this to get statistcs
        (time on first line of treatment, number of switches, stops)
    """

    df.index.name = 'pid_index'  # avoid errors, and yes, require pid to be in index (?)

    df = df.dropna(subset=[pid, event_start])

    if first_date:
        df = df.dropna(subset=[first_date])

        # if a column is specified
        if first_date in df.columns:
            include = (df[event_start] >= df[first_date])
            # if a single overall date is specified
        else:
            date = pd.to_datetime(first_date)
            include = (df[event_start] >= date)
        df = df[include]

    if last_date:
        df = df.dropna(subset=[last_date])

        if last_date in df.columns:
            include = (df[event_start] <= df[last_date])
        else:
            date = pd.to_datetime(last_date)
            include = (df[event_start] <= df[last_date])
        df = df[include]

    # period represents the days from the first_date to be included
    # cannot specify both period and last_date(?)
    if period:
        if first_date:
            end_date = df[first_date] + pd.to_timedelta(period, unit='D')
            include = (df[event_start] <= end_date)
        else:
            time_after = (df[event_start] - df.groupby(pid)[event_start].min()) / np.timedelta64(1, 'D')
            include = (time_after <= period).values  # strange need this, tries to reindex if not
        df = df[include]

    # fix formatting of input
    if _fix:
        df, cols = _to_df(df=df, cols=cols)
        codes, cols, allcodes, sep = _fix_args(df=df, codes=codes, cols=cols, sep=sep)
    else:
        allcodes=_get_allcodes(codes)

    # get the rows with the relevant columns
    rows = get_rows(df=df, codes=allcodes, cols=cols, sep=sep, _fix=False)
    subset = df[rows]  # do I need to copy?
    subset.index.name = 'pid_index'
    subset = subset.sort_values(by=[pid, event_start]).set_index('pid')

    # extract relevant codes and aggregate for each person
    code_series = extract_codes(df=subset, codes=codes, cols=cols, sep=sep, new_sep='', merge=True, out='text',
                                _fix=False)
    #    if isinstance(code_series, pd.DataFrame):
    #        code_series = pd.Series(code_series)
    string_df = code_series.groupby(level=0).apply(lambda codes: codes.str.cat(sep=time_sep))

    # eliminate repeats in string
    if not keep_repeats:
        string_df = string_df.str.replace(r'([a-z])\1+', r'\1')

    if only_unique:
        def uniqify(text):
            while re.search(r'([a-z])(.*)\1', text):
                text = re.sub(r'([a-z])(.*)\1', r'\1\2', text)
            return text

        string_df = string_df.apply(uniqify)
    return string_df

In [ ]:
#export
def del_repeats(str_series):
    """
    deletes consecutively repeated characters from the strings in a series

    """
    no_repeats = str_series.str.replace(r'([a-z])\1+', r'\1')
    return no_repeats

In [ ]:
#export
def del_singles(text):
    """
    Deletes single characters from string
    todo: how to deal with first and last position ... delete it too?

    """
    # text with only one character are by definition singles
    if len(text) < 2:
        no_singles = ''
    else:
        no_singles = "".join([letter for n, letter in enumerate(text[1:-1], start=1) if
                              ((text[n - 1] == letter) or (text[n + 1] == letter))])
        # long textx may not have any singles, so check before continue
        if len(no_singles) < 1:
            no_singles = ''
        else:
            if text[0] == no_singles[0]:
                no_singles = text[0] + no_singles
            if text[-1] == no_singles[-1]:
                no_singles = no_singles + text[-1]

    return no_singles

In [ ]:
#export
def stringify_time(df,
                   codes=None,
                   cols=None,
                   pid='pid',
                   sep=None,
                   step=90,

                   event_start='date',  # use start end
                   nfirst=None,  # ncodes

                   first_date=None,
                   # use just first, last, censored. Accept integers to indicate period/days relative to the start date
                   last_date=None,
                   censored_date=None,

                   time_sep='|',
                   no_event=' ',
                   collision='*',

                   merge=True,
                   info=None):
    """
    Creates a string for each individual describing events at position in time

    Args:
        df: dataframe
        codes: codes to be used to mark an event
        cols: columns with the event codes
        pid: column with the personal identification number
        event_start: column containing the date for the event
        sep: the seperator used between events if a column has multiple events in a cell
        keep_repeats: identical events after each other are reduced to one (if true)
        only_unique: deletes all events that have occurred previously for the individual (if true)

    Returns:
        series with a string that describes the events for each individual

    Example:
        codes={'i': '4AB02', 'a':'4AB04'}
        codes={'i': ['4AB02','L04AB02'], 'a': ['4AB04', 'L04AB04'], 'e':['4AB01']}


        df['diagnosis_date']=df[df.icdmain.fillna('').str.contains('K50|K51')].groupby('pid')['start_date'].min()

    a=stringify_time(df=mdf,  codes=codes, cols='codes', pid='pid', event_start='date',
    first_date='first_ibd', step=90, sep=',', no_event=' ', time_sep=' ')


    background
        to identify treatment patters, first stringify each treatment,
        then aggregate the different treatments to one string
        each "cell" in the string (separated by sep) represent one time unit
        the time unit can be further aggregated to reduce the level of detail

    example output (one such row for each person)
        a---s, a---, ai-s, a---, ----

        Interpretation: A person with event a and s in first time perod, then a only in second,
        the a, i and s in the third, a only in fourth and no events in the last

    purpose
        examine typical treatment patterns and correlations
        use regex or other string operations on this to get statistcs
        (time on first line of treatment, number of switches, stops)
    """

    # drop rows with missing observations in required variables
    df = df.dropna(subset=[pid, event_start])

    # find default min and max dates to be used if not user specified
    min_date = df[event_start].min()
    max_date = df[event_start].max()

    # drop rows outside time period of interest
    if first_date:
        if first_date in df.columns:
            df = df[df[event_start] >= df[first_date]]
        else:
            min_date = pd.to_datetime(first_date)
            df = df[df[event_start] >= min_date]

    if last_date:
        if last_date in df.columns:
            df = df[df[event_start] >= df[last_date]]
        else:
            max_date = pd.to_datetime(last_date)
            df = df[df[event_start] <= max_date]

    # note an individual min date cannot be before overall specified min date
    # should raise error if user tries this
    # same with max: individual cannot be larger than overall

    max_length_days = (max_date - min_date).days
    max_length_steps = int(max_length_days / step)

    # if codes or nfirst are not specified, use the five most common codes
    if not codes:
        cols = _expand_cols(_listify(cols))
        if not nfirst: nfirst = 5
        codes = count_codes(df=df, cols=cols, sep=sep).sort_values(ascending=False)[:nfirst]

    # fix formatting of input (make list out of a string input and so on)
    codes, cols, allcodes, sep = _fix_args(df=df, codes=codes, cols=cols, sep=sep)

    # get the rows that contain the relevant codes
    rows = get_rows(df=df, codes=allcodes, cols=cols, sep=sep, _fix=False)
    subset = df[rows].copy()  # maybe use .copy to avoid warnings?
    subset.index.name = 'pid_index'

    # find position of each event (number of steps from overall min_date)
    subset['position'] = (subset[event_start] - min_date).dt.days.div(step).astype(int)

    subset = subset.sort_values(by=[pid, 'position']).set_index([pid, 'position'])

    # create series with only the relevant codes for each person and position
    code_series = extract_codes(df=subset,
                                codes=codes,
                                cols=cols,
                                sep=sep,
                                new_sep=',',
                                merge=True,
                                out='text',
                                _fix=False)

    # base further aggregation on the new extracted series with its col and codes
    col = code_series.name
    codes = code_series.name.split(', ')

    # drop duplicates (same type of even in same period for same individual)
    code_series = code_series.reset_index().drop_duplicates().set_index(pid, drop=False)
    code_series.index.name = 'pid_index'

    ## make dict with string start end end positions for each individual
    # explanation:
    # the string is first made marking events in positions using calendar time
    # but often we want the end result to be strings that start at specified
    # individual dates, and not the same calendar date for all
    # for instance it is often useful to start the string at the date the
    # person receives a diagnosis
    # same with end of string: strings may end when a patient dies
    # user can specify start and end dates by pointing to columns with dates
    # or they may specify an overall start and end date
    # if individual dates are specified, the long string based on calendar
    # time is sliced to include only the relevant events

    if first_date:
        # if a column is specified
        if first_date in subset.columns:
            start_date = subset.groupby(pid)[first_date].first().dropna().to_dict()
        # if a single overall date is specified
        else:
            date = pd.to_datetime(first_date)
            start_date = {pid: date for pid in subset[pid].unique()}
        # convert start date to start position in string
        start_position = {pid: int((date - min_date).days / step)
                          for pid, date in start_date.items()}

    if last_date:
        if last_date in subset:
            end_date = subset.groupby(pid)[last_date].first().dropna().to_dict()
        else:
            date = pd.to_datetime(last_date)
            end_date = {pid: date for pid in subset[pid].unique()}
        # convert date to position in string
        end_position = {pid: (date - min_date).dt.days.div(step).astype(int)
                        for pid, date in end_date.items()}

    # takes dataframe for an individual and makes a string with the events
    def make_string(events):
        # get pid of individual (required to find correct start and end point)
        person = events[pid].iloc[0]

        # make a list of maximal length with no events
        event_list = [no_event] * (max_length_steps + 1)

        # loop over all events the individual has and put code in correct pos.
        for pos in events['position'].values:
            event_list[pos] = code

        event_string = "".join(event_list)

        # slice to correct start and end of string (if specified)
        if first_date:
            event_string = event_string[start_position[person]:]
        if last_date:
            event_string = event_string[:-(max_length_steps - end_position[person])]
        return event_string

    # new dataframe to store each string for each individual for each code
    string_df = pd.DataFrame(index=code_series[pid].unique())

    # loop over each code, create aggregate string for each individual, store in df
    for code in codes:
        code_df = code_series[code_series[col].isin([code])]
        stringified = code_df.groupby(pid, sort=False).apply(make_string)
        string_df[code] = stringified

    if merge:
        string_df = interleave_strings(string_df, no_event=no_event, time_sep=time_sep)
    return string_df

# formatting an expression

## insert_external

In [0]:
#export
def insert_external(expr):
  """
  Replaces variables prefixed with @ in the expression with the
  value of the variable from the global namespace

  Example:
      x=['4AB02', '4AB04', '4AB06']
      expr = '@x before 4AB02'
      insert_external(expr)
  """
  externals = [word.strip('@') for word in expr.split() if word.startswith('@')]
  for external in externals:
      tmp = globals()[external]
      expr = expr.replace(f'@{external} ', f'{tmp} ')
  return expr

In [0]:
  x_1=['4AB02', '4AB04', '4AB06']
  expr = '@x_1 before 4AB02'
  insert_external(expr)

# get inpatient data

To test the functions and to calculate the Charslon index we need some data. Here we will use data on hospital visits from Medicare: 


In [0]:
# Use pandas
import pandas as pd

In [0]:
# Read synthetic medicare sample data on inpatient hospital stays
path = 'https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/SynPUFs/Downloads/'
inpatient_file = 'DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.zip'

inpatient = pd.read_csv(path+inpatient_file)

inpatient.columns = inpatient.columns.str.lower()
# easier to use a column called 'pid' than 'desynpuf_id'
inpatient['pid']=inpatient['desynpuf_id']

#set index to the personal id, but also keep id as a column
inpatient = inpatient.set_index('pid', drop=False)
inpatient.index.name='pid_index'


In [0]:
# Have a look
inpatient.head()

In [0]:
# make a list of columns with information about diagnostic codes
icd_cols = list(inpatient.columns[inpatient.columns.str.startswith('icd9_dgns_cd')])
icd_cols

Make a list of all unique ICD9 codes that exist, a all_codes:

In [0]:
# Codes to calculate CCI using ICD-9 (CM, US, Enhanced)
# Source: http://mchp-appserv.cpe.umanitoba.ca/concept/Charlson%20Comorbidities%20-%20Coding%20Algorithms%20for%20ICD-9-CM%20and%20ICD-10.pdf

infarction = '''
      410* 
      412*
      '''

heart_failure = '''
        390.91 
        402.21 402.11 402.91 
        404.01 404.03 404.11 404.13 404.91 404.93 
        425.4-425.9 
        428*
        '''

peripheral_vascular = '''
        093.0
        437.3
        440*
        441*
        443.1-443.9
        447.1
        557.1 557.9
        V43.4
        '''

cerebrovascular = '''
        362.34
        430*-438*
        '''
dementia = '''
        290*
        294.1
        331.2
        '''

pulmonary ='''
      416.8 416.9
      490*-505* 
      506.4
      508.1 508.8
      '''
rheumatic = '''
      446.5
      710.0-710.4
      714.0-714.2 714.8
      725*
      '''

peptic_ulcer = '531*-534*'

liver_mild ='''
      070.22
      070.23
      070.32
      070.33
      070.44
      070.54
      070.6
      070.9
      570.*
      571.*
      573.3 573.4 573.8 573.9
      V42.7
      '''
# Interesting, diabetes seems to be 5 digits long in the data, but not the specified codes
diabetes_without_complication = '250.0*-250.3* 250.8* 250.9*'

diabetes_with_complication = '250.4*-250.7*'

plegia = '''
    334.1
    342.*
    343.*
    344.0-344.6
    344.9
    '''

renal = '''
    403.01 403.11,403.91 
    404.02 404.03 404.12 404.13 404.92 404.93
    582.*  
    583.0-583.7
    585*
    586*
    588.0
    V42.0
    V45.1
    V56*
    '''

malignancy = '''
    140*-172*
    174.0-195.8
    200*-208*
    238.6
    '''

liver_not_mild = '''
    456.0-456.2
    572.2-572.8
    '''

tumor = '196*-199*'

hiv = '042*-044*'

Put all the strings that describe the codes for the comorbitities in a single datastructure:

In [0]:
icd9 = unique(df=inpatient, cols = icd_cols, all_str=True)

In [0]:
# A dictionary with names of cormobitities and the associated medical codes

code_string = { 'infarction' : infarction, 
               'heart_failure' : heart_failure, 
               'peripheral_vascular' : peripheral_vascular, 
               'cerebrovascular' : cerebrovascular, 
               'dementia' : dementia, 
               'pulmonary' : pulmonary, 
               'rheumatic' : rheumatic, 
               'peptic_ulcer' : peptic_ulcer, 
               'liver_mild' : liver_mild, 
               'diabetes_without_complication' : diabetes_without_complication, 
               'diabetes_with_complication' : diabetes_with_complication, 
               'plegia' : plegia, 
               'renal' : renal, 
               'malignancy' : malignancy, 
               'liver_not_mild' : liver_not_mild, 
               'tumor' : tumor, 
               'hiv' : hiv}

Having created a all_codes, we can use the functions we have created to expand the description for all the different comorbidities to include all the specific codes:

In [0]:
codes = {disease : expand_code(codes.split(), 
                               all_codes=icd9,
                               drop_dot=True,
                               drop_leading_zero=True) 
        for disease, codes in code_string.items()}

And we can check if it really expanded the codes, for instance by examining the codes for mild liver disease:

In [0]:
codes['liver_mild']

In order to do the calculations, we need the weights associated with each comorbidity. These weights are related to the predictive power of the comorbididy for the probability of dying in a given time period. There are a few different standards, but with relatively minor varitions. Here we use the following:


In [0]:
charlson_points = { 'infarction': 1, 
                   'heart_failure': 1, 
                   'peripheral_vascular': 1, 
                   'cerebrovascular': 1, 
                   'dementia': 1, 
                   'pulmonary': 1, 
                   'rheumatic': 1, 
                   'peptic_ulcer': 1, 
                   'liver_mild': 1, 
                   'diabetes_without_complication': 1, 
                   'diabetes_with_complication': 2, 
                   'plegia': 2, 
                   'renal': 2, 
                   'malignancy': 2, 
                   'liver_not_mild': 3, 
                   'tumor': 6, 
                   'hiv': 6}


We also need the function that takes a set of codes and identifies the rows and persons who have the codes (a function we developed in a previous notebook):

In [ ]:
#hide
from nbdev.showdoc import *

In [1]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
Converted query_language.ipynb.


In [2]:
import nbdev

In [3]:
from nbdev.sync import script2notebook 

In [ ]:
script2not